In [1]:
# Imports
import pandas as pd
import datasets
import transformers
import csv

from IPython.display import display, HTML
from datasets import ClassLabel

In [2]:
# Test CUDA
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

print("Device:", device)
print("Version:", torch.__version__)

Device: cuda
Version: 1.8.1


In [3]:
# Load swiss data
path_corpus = "data_train.csv"

data_txt = []
data_ref = []

with open("data_train.csv", "r", encoding="utf-8") as f:
    reader = csv.reader(f, delimiter=",", quoting=csv.QUOTE_ALL)
    next(reader, None)

    for row in reader:
        data_txt.append(row[0])
        data_ref.append(row[1])

tuples = list(zip(data_txt, data_ref))
dataframe = pd.DataFrame(tuples, columns=["article", "highlights"])

In [4]:
# Load german data
tuples = pd.read_excel("data_train_test.xlsx", engine="openpyxl")
del tuples["Unnamed: 0"]
dataframe = pd.concat([dataframe, tuples])
dataframe = dataframe.dropna()

In [5]:
# Clean redactional data
print(len(dataframe))
dataframe = dataframe[~dataframe["highlights"].str.contains("ZEIT")]
print(len(dataframe))

112491
112405


In [6]:
# Concat swiss and german data
german_data = datasets.arrow_dataset.Dataset.from_pandas(dataframe[["article", "highlights"]])
german_data = german_data.shuffle()
print(dataframe)

                                                 article  \
0      Minghella war der Sohn italienisch-schottische...   
1      Ende der 1940er Jahre wurde eine erste Auteur-...   
2      Al Pacino, geboren in Manhattan, ist der Sohn ...   
3      Der Name der Alkalimetalle leitet sich von dem...   
4      Die Arbeit ist bereits seit dem Altertum Gegen...   
...                                                  ...   
12533  Das in der Kritik stehende Kommando Spezialkrä...   
12534  Franka Lu ist eine chinesische Journalistin un...   
12535  Nach drei Angriffen mit explosiven Postsendung...   
12536  Russische Behörden haben nach eigenen Angaben ...   
12537  DIE ZEIT: Herr Heisterhagen, Ralf Stegner kand...   

                                              highlights  
0      Anthony Minghella, CBE war ein britischer Film...  
1      Die Auteur-Theorie ist eine Filmtheorie und di...  
2      Alfredo James "Al" Pacino ist ein US-amerikani...  
3      Als Alkalimetalle werden die chemisc

In [7]:
# Split data
train_size = int(len(dataframe) * 0.7)
valid_size = int(len(dataframe) * 0.2)
test_size = int(len(dataframe) * 0.1)

train_data = german_data.select(range(0, train_size))
val_data = german_data.select(range(train_size, train_size + valid_size))
test_data = german_data.select(range(train_size + valid_size, len(dataframe)))

In [8]:
# Load english data
'''
train_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")
val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:5%]")
test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test[:2%]")
'''

'\ntrain_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")\nval_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:5%]")\ntest_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test[:2%]")\n'

In [9]:
# Reduce iteration corpus
train_data = train_data.shuffle()
train_data = train_data.select(range(0, 10000))

val_data = val_data.shuffle()
val_data = val_data.select(range(0, 2000))

test_data = test_data.shuffle()
test_data = test_data.select(range(0, 2000))

In [10]:
# Explore corpus
df = pd.DataFrame(train_data)

text_list = []
summary_list = []

for index, row in df.iterrows():
    text = row["article"]
    summary = row["highlights"]
    text_list.append(len(text))
    summary_list.append(len(summary))
    
print(sum(text_list) / len(text_list))
print(sum(summary_list) / len(summary_list))

4037.716
242.733


In [11]:
# Explore corpus
train_data.info.description
df = pd.DataFrame(train_data[:1])

for column, typ in train_data.features.items():
    if isinstance(typ, ClassLabel):
        df[column] = df[column].transform(lambda i: typ.names[i])

display(HTML(df.to_html()))

,article,highlights,__index_level_0__
0,"Wiktor Schuwalow begann seine Karriere in der Saison 1947/48 bei Dserschinez Tscheljabinsk, ehe er zur Saison 1949/50 zum Verein der Luftstreitkräfte WWS MWO Moskau delegiert wurde. Mit WWS gewann er drei Sowjetische Meisterschaften und eine Vizemeisterschaft, ehe der Verein im zuge der Entstalinisierung aufgelöst wurde. Anschliessend spielte er für den zentralen Armeesportklub ZDSA Moskau, mit dem er drei Pokalsiege feierte. Zwischen 1957 und 1959 liess er seine Karriere beim SKA MWO Kalinin als Spielertrainer ausklingen. Insgesamt erzielte er 222 Tore in 150 Spielen in der höchsten sowjetischen Liga. Schuwalow spielte, wie viele andere Eishockeyspieler auch, während des Sommers Fussball. Für WWS MWO Moskau lief er zwischen 1950 und 1952 75 Mal in der höchsten Fussball-Spielklasse der Sowjetunion auf und erzielte dabei 25 Tore. Am 29. Januar 1954 stand er in einem Spiel gegen Finnland zum ersten Mal für die sowjetische Eishockeynationalmannschaft auf dem Eis. 1953 wurde er als Verdienter Meister des Sports ausgezeichnet. Seine internationale Karriere wurde mit der Goldmedaille bei den Olympischen Winterspielen 1956 gekrönt. Für die Nationalmannschaft erzielte er 40 Tore in 51 Länderspielen. Am 3. Januar 1957 bestritt er sein letztes Länderspiel. 2004 wurde er in die Ruhmeshalle des russischen Eishockeyverbands aufgenommen. 2014 erhielt er in Anerkennung für den Olympiasieg 1956 den Orden der Ehre. Nachdem er den SKA MWO Kalinin bereits als Spielertrainer betreut hatte, war Schuwalow anschliessend bis 1963 Cheftrainer der Mannschaft. Zwischen 1964 und 1968 betreute er in gleicher Funktion die erste Mannschaft von Kristall Elektrostal. Anschliessend wechselte er zu Spartak Moskau und arbeitete dort in der Saison 1968/69 als Assistenztrainer. 1970 war er Trainer des Nachwuchsvereins ""DJuSSch"" von Chimik Woskressensk, anschliessend bis 1973 hauptamtlicher Trainer des UdSSR-Sportausschuss für Eishockey. Zwischen 1973 und 1975 war Schuwalow Cheftrainer der rumänischen Nationalmannschaft, ehe er zu den Junioren von Chimik zurückkehrte. Zwischen 1978 und 1984 war er Direktor des Nachwuchszentrums ""SDJuSCHOR"" von Spartak Moskau.",Wiktor Grigorjewitsch Schuwalow ist ein ehemaliger sowjetisch-russischer Eishockeyspieler und -trainer. Mit WWS MWO Moskau gewann er dreimal die Sowjetische Meisterschaft. Der grösste Erfolg seiner Karriere war der Olympiasieg 1956.,34375


In [12]:
# Load tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-multilingual-cased")
print(type(tokenizer))

<class 'transformers.models.bert.tokenization_bert.BertTokenizer'>


In [13]:
# Prepare data
encoder_max_length = 512
decoder_max_length = 128
batch_size = 2 # 16

def process_data_to_model_inputs(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=decoder_max_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["decoder_attention_mask"] = outputs.attention_mask
    batch["labels"] = outputs.input_ids.copy()
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

    return batch

In [14]:
# Training data
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights"] # id
)

train_data.set_format(
    type="torch",
    columns=["input_ids",
             "attention_mask",
             "decoder_input_ids",
             "decoder_attention_mask",
             "labels"]
)

In [15]:
# Validation data
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    remove_columns=["article", "highlights"] # id
)

val_data.set_format(
    type="torch",
    columns=["input_ids",
             "attention_mask",
             "decoder_input_ids",
             "decoder_attention_mask",
             "labels"]
)

In [16]:
# Load models
tf2tf = transformers.EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-multilingual-cased", "bert-base-multilingual-cased", tie_encoder_decoder=False)
tf2tf.save_pretrained("bert2bert_multilingual")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias

In [31]:
# Configure models
tf2tf.config.decoder_start_token_id = tokenizer.cls_token_id
tf2tf.config.eos_token_id = tokenizer.sep_token_id
tf2tf.config.pad_token_id = tokenizer.pad_token_id
tf2tf.config.vocab_size = tf2tf.config.encoder.vocab_size

print(tf2tf.config.decoder_start_token_id)
print(tf2tf.config.eos_token_id)
print(tf2tf.config.vocab_size)

101
102
119547


In [18]:
# Configure beam search
tf2tf.config.max_length = 142
tf2tf.config.min_length = 56
tf2tf.config.no_repeat_ngram_size = 3
tf2tf.config.early_stopping = True
tf2tf.config.length_penalty = 2.0
tf2tf.config.num_beams = 4

In [19]:
# Prepare metric
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [20]:
# Load checkpoint
path_output = "./"
path_checkpoint = path_output + "/bert2bert_multilingual"

tf2tf = transformers.EncoderDecoderModel.from_pretrained(path_checkpoint)
tf2tf.to("cuda")

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [21]:
# Empty cache
import gc
import psutil

gc.collect()
torch.cuda.empty_cache()
psutil.virtual_memory()

svmem(total=67274498048, available=36111126528, percent=46.3, used=30709243904, free=31127035904, active=31459164160, inactive=1514299392, buffers=19013632, cached=5419204608, shared=176472064, slab=879636480)

In [22]:
# Setup arguments
training_args = transformers.Seq2SeqTrainingArguments(
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    output_dir=path_output,
    warmup_steps=1000,
    save_steps=2000,
    logging_steps=1000,
    eval_steps=2000,
    save_total_limit=1,
    fp16=True,
    # predict_with_generate=True
)

In [23]:
# Start training
trainer = transformers.Seq2SeqTrainer(
    model=tf2tf,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data
)

trainer.train()

/home/davo557d/user-kernel-3/test/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss,Validation Loss


TrainOutput(global_step=1500, training_loss=4.432283284505209, metrics={'train_runtime': 2251.704, 'train_samples_per_second': 0.666, 'total_flos': 4425924222720000.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 62656, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 739894, 'train_mem_gpu_alloc_delta': 4671432192, 'train_mem_cpu_peaked_delta': 979164, 'train_mem_gpu_peaked_delta': 2829857280})

In [ ]:
# Load checkpoint
path_output = "./"
path_checkpoint = path_output + "/bert2bert_multilingual"

tf2tf = EncoderDecoderModel.from_pretrained(path_checkpoint)
tf2tf.to("cuda")

In [33]:
# Evaluate training
def generate_summary(batch):
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")
    
    outputs = tf2tf.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    batch["pred_summary"] = output_str

    return batch

results = test_data.map(
    generate_summary,
    batched=True,
    batch_size=batch_size
)

print(f"HYP: {results[0]['pred_summary']}")
print(f"REF: {results[0]['highlights']}")

rouge.compute(predictions=results["pred_summary"], references=results["highlights"], rouge_types=["rouge2"])["rouge2"].mid


HYP: Die ist eine in der. Sie wurde am 1. Mai 1990 als
REF: Die Letov LF-107 "Luňák" ist ein leistungsfähiges tschechoslowakisches Kunstflug-Segelflugzeug, das in den späten 1940er Jahren von Rudý Letov entwickelt und gebaut wurde.


Score(precision=0.0780251512376513, recall=0.038187921078750985, fmeasure=0.045264512047422364)